# Урок 9. Python применение продвинутых методов
Оцените результаты эксперимента :
Стат значимо ли отличается метрика конверсии в двух группах ? Какие рекомендации дадите менеджеру? 
  
About this file  
For this project, you will be working to understand the results of an A/B test run by an e-commerce website. The company has developed a new web page in order to try and increase the number of users who "convert," meaning the number of users who decide to pay for the company's product. Your goal is to work through this notebook to help the company understand if they should implement this new page, keep the old page, or perhaps run the experiment longer to make their decision 
  
Об этом файле  
В этом проекте вы будете работать над тем, чтобы понять результаты A/B-тестирования, проведенного веб-сайтом электронной коммерции. Компания разработала новую веб-страницу, чтобы попытаться увеличить количество пользователей, которые «конвертируют», то есть количество пользователей, которые решили заплатить за продукт компании. Ваша цель — проработать этот блокнот, чтобы помочь компании понять, следует ли ей внедрить эту новую страницу, оставить старую страницу или, возможно, провести эксперимент дольше, чтобы принять решение..ts

*__Подготовка и анализ начальных данных.__*

In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('a_b test ecom.xlsx')

data.head(10)

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
5,936923,20:49.1,control,old_page,0
6,679687,26:46.9,treatment,new_page,1
7,719014,48:29.5,control,old_page,0
8,817355,58:09.0,treatment,new_page,1
9,839785,11:06.6,treatment,new_page,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         294478 non-null  int64 
 1   time       294478 non-null  object
 2   con_treat  294478 non-null  object
 3   page       294478 non-null  object
 4   converted  294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [4]:
data.isna().sum()

id           0
time         0
con_treat    0
page         0
converted    0
dtype: int64

In [5]:
data.con_treat.value_counts()

con_treat
treatment    147276
control      147202
Name: count, dtype: int64

__*Избавимся от некорретных строк.*__

_Посчитаем, сколько типов встречается у каждого пользователя._

In [6]:
v = data.\
    groupby('id', as_index=False).\
    agg({'con_treat': pd.Series.nunique})

v.head()

,id,con_treat
0,630000,1
1,630001,1
2,630002,1
3,630003,1
4,630004,1


In [7]:
v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290584 entries, 0 to 290583
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   id         290584 non-null  int64
 1   con_treat  290584 non-null  int64
dtypes: int64(2)
memory usage: 4.4 MB


In [8]:
more_than_one_types = v.query('con_treat > 1')
data_new = data[~data.id.isin(more_than_one_types.id)].sort_values('id')
data_new.shape

(290688, 5)

Уже видно, что почти 4 тысячи юзеров попали в обе группы.

_Далее проверим правильность сплитования_

In [9]:
len(data_new[(data_new['con_treat']=='control') & (data_new['page']=='new_page')])

1007

In [10]:
len(data_new[(data_new['con_treat']=='treatment') & (data_new['page']=='old_page')])

991

Ещё 1998 юзеров не соответствуют сплитованным группам (т.е. в контрольной группе с тестовой страницей и наоборот)

In [11]:
data_final = data_new.loc[(data_new['con_treat']=='control')&(data_new['page']=='old_page') | (data_new['con_treat']=='treatment')&(data_new['page']=='new_page')]
data_final.shape

(288690, 5)

Итого отсеялось 5788 юзеров.  Это вызывает вопросы к сплитовалке.

In [12]:
data_final.con_treat.value_counts()

con_treat
treatment    144390
control      144300
Name: count, dtype: int64

Различия в размерах выборок незначительны.

__*Для проверки конверсии будем использовать z-критерий*__

In [13]:
control = data_final.query("con_treat == 'control'")
treatment = data_final.query("con_treat == 'treatment'")

n_control = control.shape[0]
n_treatment = treatment.shape[0]

conv_control = control['converted'].sum()
conv_treatment = treatment['converted'].sum()

In [14]:
from statsmodels.stats.proportion import proportions_ztest

z_stat, p_value = proportions_ztest([conv_control, conv_treatment], [n_control, n_treatment])

print(f"Z-statistic: {z_stat:.2f}\nP-value: {p_value:.4f}")

Z-statistic: 1.22
P-value: 0.2223


### Итог

__pvalue > 0.05__  
__*Статистически значимой разницы между выборками не обнаружено. Остаёмся на контрольной/начальной версии сайта.*__  
__*Необходимо проверить сплитовалку*__